In [ ]:
import streamlit as st
import pinecone
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Initialize Pinecone
pinecone.init(api_key='3d87982d-aa12-4905-aa83-976f592e8c1b', environment='us-east-1-aws')
index_name = 'coreytest'

# Connect to the Pinecone index
index = pinecone.Index(index_name)

# Load the embedding model
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# Function to embed user input
def embed_ingredients(ingredients):
    ingredients_str = ' '.join(ingredients)
    embedding = model.encode(ingredients_str, convert_to_tensor=True)
    return embedding

# Streamlit UI
st.title("Recipe Finder")
st.write("Enter ingredients to find the best matching recipes:")

# Input ingredients
ingredients_input = st.text_area("Ingredients (comma-separated)", "tomato, basil, garlic")

if st.button("Find Recipes"):
    if ingredients_input:
        ingredients_list = [ingredient.strip() for ingredient in ingredients_input.split(',')]
        user_embedding = embed_ingredients(ingredients_list)
        
        # Query Pinecone to get embeddings and metadata
        results = index.query(user_embedding.cpu().numpy().tolist(), top_k=5, include_metadata=True)

        # Process results
        st.write("Here are the best matching recipes:")
        for match in results['matches']:
            recipe = match['metadata']
            st.write(f"### {recipe['name']}")
            st.write(f"[Link to recipe]({recipe['url']})")
            st.write(f"**Ingredients:** {recipe['ingredients']}")
            st.write(f"**Score:** {match['score']}")
            st.write("---")

# Running Streamlit app
if __name__ == "__main__":
    st._is_running_with_streamlit = True
    st.run()
